In [1]:
import numpy as np

def random_initialization(A, rank):
    """
    Initialize matrices W and H randomly.

    Parameters:
    - A: Input matrix
    - rank: Rank of the factorization

    Returns:
    - W: Initialized W matrix
    - H: Initialized H matrix
    """
    num_docs = A.shape[0]
    num_terms = A.shape[1]
    W = np.random.uniform(1, 2, (num_docs, rank))
    H = np.random.uniform(1, 2, (rank, num_terms))
    return W, H

def nndsvd_initialization(A, rank):
    """
    Initialize matrices W and H using Non-negative Double Singular Value Decomposition (NNDSVD).

    Parameters:
    - A: Input matrix
    - rank: Rank of the factorization

    Returns:
    - W: Initialized W matrix
    - H: Initialized H matrix
    """
    u, s, v = np.linalg.svd(A, full_matrices=False)
    v = v.T
    w = np.zeros((A.shape[0], rank))
    h = np.zeros((rank, A.shape[1]))

    w[:, 0] = np.sqrt(s[0]) * np.abs(u[:, 0])
    h[0, :] = np.sqrt(s[0]) * np.abs(v[:, 0].T)

    for i in range(1, rank):
        ui = u[:, i]
        vi = v[:, i]
        ui_pos = (ui >= 0) * ui
        ui_neg = (ui < 0) * -ui
        vi_pos = (vi >= 0) * vi
        vi_neg = (vi < 0) * -vi

        ui_pos_norm = np.linalg.norm(ui_pos, 2)
        ui_neg_norm = np.linalg.norm(ui_neg, 2)
        vi_pos_norm = np.linalg.norm(vi_pos, 2)
        vi_neg_norm = np.linalg.norm(vi_neg, 2)

        norm_pos = ui_pos_norm * vi_pos_norm
        norm_neg = ui_neg_norm * vi_neg_norm

        if norm_pos >= norm_neg:
            w[:, i] = np.sqrt(s[i] * norm_pos) / ui_pos_norm * ui_pos
            h[i, :] = np.sqrt(s[i] * norm_pos) / vi_pos_norm * vi_pos.T
        else:
            w[:, i] = np.sqrt(s[i] * norm_neg) / ui_neg_norm * ui_neg
            h[i, :] = np.sqrt(s[i] * norm_neg) / vi_neg_norm * vi_neg.T

    return w, h

def multiplicative_update(A, k, max_iter, init_mode='random'):
    """
    Perform Multiplicative Update (MU) algorithm for Non-negative Matrix Factorization (NMF).

    Parameters:
    - A: Input matrix
    - k: Rank of the factorization
    - max_iter: Maximum number of iterations
    - init_mode: Initialization mode ('random' or 'nndsvd')

    Returns:
    - W: Factorized matrix W
    - H: Factorized matrix H
    - norms: List of Frobenius norms at each iteration
    """
    if init_mode == 'random':
        W, H = random_initialization(A, k)
    elif init_mode == 'nndsvd':
        W, H = nndsvd_initialization(A, k)

    norms = []
    epsilon = 1.0e-10
    for _ in range(max_iter):
        # Update H
        W_TA = W.T @ A
        W_TWH = W.T @ W @ H + epsilon
        H *= W_TA / W_TWH

        # Update W
        AH_T = A @ H.T
        WHH_T = W @ H @ H.T + epsilon
        W *= AH_T / WHH_T

        norm = np.linalg.norm(A - W @ H, 'fro')
        norms.append(norm)

    return W, H, norms


Let's try out the methods.

In [3]:
import numpy as np
# Generate a random matrix A
A = np.random.rand(10, 10)
print(" A : " , A )
# Test random_initialization
rank = 3
W_rand, H_rand = random_initialization(A, rank)
print("Random Initialization:")
print("W:\n", W_rand)
print("H:\n", H_rand)
print()

# Test nndsvd_initialization
W_nndsvd, H_nndsvd = nndsvd_initialization(A, rank)
print("NNDSVD Initialization:")
print("W:\n", W_nndsvd)
print("H:\n", H_nndsvd)
print()

# Test multiplicative_update
max_iter = 100
W_mu, H_mu, norms = multiplicative_update(A, rank, max_iter)

print("Multiplicative Update:")
print("W:\n", W_mu)
print("H:\n", H_mu)
print("Frobenius Norms:\n", norms)


 A :  [[2.23988670e-01 9.08856228e-01 2.30754876e-01 5.61795374e-01
  9.18707054e-04 5.59852596e-01 7.87258900e-01 5.14913742e-01
  1.37083176e-01 1.26652406e-01]
 [3.39093026e-01 5.85620763e-01 4.64536805e-01 8.65045660e-01
  1.49156805e-01 5.73586873e-01 1.10505686e-01 2.26131564e-01
  7.56064845e-01 4.43944053e-01]
 [2.98547131e-01 7.05966817e-01 2.05161565e-01 8.13246881e-02
  7.07874063e-01 6.54163113e-01 3.15848687e-01 8.36060161e-02
  9.94348785e-01 1.34157567e-01]
 [4.12495357e-01 7.93156303e-01 7.24550476e-01 4.99878365e-01
  5.91496442e-01 3.14655562e-01 5.97981804e-01 3.89668916e-01
  6.43510831e-01 3.59729977e-01]
 [8.51198606e-01 4.79491278e-01 1.42761109e-01 7.68008959e-01
  3.22672209e-01 3.32934793e-01 4.69221569e-01 4.16985506e-02
  7.59146374e-03 6.78967203e-01]
 [2.58020163e-01 6.37295292e-01 2.98469422e-01 3.45447838e-01
  6.58441384e-01 4.46607229e-01 8.25798863e-01 1.08752388e-01
  6.02383052e-01 4.49641908e-01]
 [2.37981656e-01 8.99057485e-01 2.03316837e-03 6.382